In [29]:

import h5py as h5
import pandas as pd
import csv
import json
import imp
from os.path import basename, splitext




In [74]:

def loadpy(fname):
    base=splitext(basename(fname))[0]
    mod=imp.load_source(base, fname)
    return mod

def loadcsv(fname):
    rows=[]
    with open(fname) as fd:
        reader = csv.reader(fd)
        for row in reader:
            rows.append(row)
    return rows

def loadjson(fname):
        with open(fname) as fd:
            return json.load(fd)
def loadh5(fname):
    return h5.File(fname)

In [75]:

ibm = loadpy("ibm.pyd")
print("exchange", ibm.EXCHANGE)



exchange NYSE


In [77]:
print("straight python")
%timeit loadpy("ibm.pyd")
print("csv")
%timeit loadcsv("ibm.csv")
print("json")
%timeit loadjson("ibm.json")
print("hdf5")
%timeit loadh5("ibm.h5")


straight python
1000 loops, best of 3: 1.77 ms per loop
csv
100 loops, best of 3: 5.3 ms per loop
json
100 loops, best of 3: 5.15 ms per loop
hdf5
The slowest run took 5.22 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 22.4 µs per loop


In [85]:
ibm = loadh5("ibm.h5")
print(ibm)

<HDF5 file "ibm.h5" (mode r+)>


In [78]:

# disadvantages to hdf5? not human readable

js = loadjson("ibm.json")

hf = h5.File("ibm.h5", "w")


ds = hf.create_dataset("rows", data=js["rows"])

# print(js["COLUMNS"])
hf.attrs['column_names']=[x.encode('ascii') for x in js["COLUMNS"]]

for key in ["EXCHANGE", "MARKET_OPEN_MINUTE", "MARKET_CLOSE_MINUTE",
           "INTERVAL", "TIMEZONE_OFFSET"]:
    print("adding %s" % key)
    value = js[key]
    if isinstance(value, str):
        hf[key]=value.encode('ascii')
    else:
        hf[key]=value
hf.close()



OSError: Unable to create file (Unable to truncate a file which is already open)